# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *
from create_tables import *

In [2]:
try:
    create_database()
except psycopg2.Error as e:
    print('Error creating database')
    print(e)

try:
    conn = psycopg2.connect("host=127.0.0.1 dbname=studentdb user=student password=student")
    cur = conn.cursor()
    drop_tables(cur, conn)
    create_tables(cur, conn)
except psycopg2.Error as e:
    print('Error dropping and creating tables')
    print(e)


In [1]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [4]:
song_files = get_files('data/song_data')

In [5]:
df = pd.DataFrame(pd.read_json(song_files[0], typ='series'))
df.head()

,0
num_songs,1
artist_id,ARNNKDK1187B98BBD5
artist_latitude,45.8073
artist_longitude,15.9676
artist_location,Zagreb Croatia


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [6]:
song_data = df[0][['song_id', 'title', 'artist_id', 'year', 'duration']].values.tolist()

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [7]:

cur.execute("SELECT table_name FROM information_schema.tables WHERE table_schema='public'", song_data)

print(cur.fetchall())
cur.execute(dim_songs_table_insert, song_data)
conn.commit()

cur.execute("SELECT * from dim_songs", song_data)
print(cur.fetchall())

[('fact_songplays',), ('dim_users',), ('dim_songs',), ('dim_artists',), ('dim_time',)]
[('SOFNOQK12AB01840FC', 'Kutt Free (DJ Volume Remix)', 'ARNNKDK1187B98BBD5', 0, Decimal('407.37914'))]


Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [8]:
artist_data = df[0][['artist_id', 'artist_name', 'artist_location', 'artist_latitude', 'artist_longitude']].values.tolist()
print(artist_data)

['ARNNKDK1187B98BBD5', 'Jinx', 'Zagreb Croatia', 45.80726, 15.967600000000001]


#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [9]:
cur.execute(dim_artists_table_insert, artist_data)
conn.commit()

cur.execute("SELECT * from dim_artists", song_data)
print(cur.fetchall())

[('ARNNKDK1187B98BBD5', 'Jinx', 'Zagreb Croatia', 45.8073, 15.9676)]


Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [10]:
log_files = get_files('data/log_data')

In [11]:
df = pd.DataFrame(pd.read_json(log_files[0], lines=True, typ='series'))
df.head()

,0
0,"{'artist': 'Sydney Youngblood', 'auth': 'Logge..."
1,"{'artist': 'Gang Starr', 'auth': 'Logged In', ..."
2,"{'artist': '3OH!3', 'auth': 'Logged In', 'firs..."
3,"{'artist': 'RÃÂ¶yksopp', 'auth': 'Logged In',..."
4,"{'artist': 'Kajagoogoo', 'auth': 'Logged In', ..."


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [12]:
df = pd.DataFrame(pd.read_json(log_files[0], lines=True, typ='series'))[0]
df.head()

0    {'artist': 'Sydney Youngblood', 'auth': 'Logge...
1    {'artist': 'Gang Starr', 'auth': 'Logged In', ...
2    {'artist': '3OH!3', 'auth': 'Logged In', 'firs...
3    {'artist': 'RÃÂ¶yksopp', 'auth': 'Logged In',...
4    {'artist': 'Kajagoogoo', 'auth': 'Logged In', ...
Name: 0, dtype: object

In [13]:
time_in_ms = df[0]['ts']
timestamp = pd.to_datetime(time_in_ms, unit='ms')

time_in_hours = timestamp.hour
time_in_days = timestamp.day
time_in_week_of_year = timestamp.week
time_in_month_of_year = timestamp.month
time_in_year = timestamp.year
time_in_weekday = timestamp.weekday()

time_data = [timestamp, time_in_hours, time_in_days, time_in_week_of_year,
             time_in_month_of_year, time_in_year, time_in_weekday]

#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [14]:
cur.execute(dim_time_table_insert, time_data)
conn.commit()

cur.execute("SELECT * from dim_time", song_data)
print(cur.fetchall())

[(datetime.datetime(2018, 11, 29, 0, 0, 57, 796000), 0, 29, 48, 11, 2018, '3')]


Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [15]:
user_df = df[0]
user_data = [user_df['userId'], user_df['firstName'], user_df['lastName'], user_df['gender'], user_df['level']]
print(user_data)

['73', 'Jacob', 'Klein', 'M', 'paid']


#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [16]:
cur.execute(dim_users_table_insert, user_data)
conn.commit()
    
cur.execute("SELECT * from dim_users", user_data)
print(cur.fetchall())

[('73', 'Jacob', 'Klein', 'M', 'paid')]


Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [24]:

for log in log_files:
    df = pd.DataFrame(pd.read_json(log, lines=True, typ='series'))
    
    for row in df[0]:
        # skip the row insert if None value is present
        if row['song'] is None or row['length'] is None or row['artist'] is None:
            continue
    
        song_data = [row['song'], row['length']]
        artist_data = [row['artist']]

        # if no song id exists, skip row insert
        song_id = get_song_id(cur, conn, song_data)

        if song_id is None:
            continue

        artist_id = get_artist_id(cur, conn, artist_data)

        # if no artist id exists, skip row insert
        if artist_id is None:
            continue

        time_in_ms = row['ts']
        timestamp = pd.to_datetime(time_in_ms, unit='ms')

        fact_songplays_data = [timestamp, row['userId'], row['level'], song_id, artist_id,
                               row['sessionId'], row['location'], row['userAgent']]

        insert_into_fact_songplays_table(cur, conn, fact_songplays_data)
    
cur.execute("SELECT * from fact_songplays", user_data)
print(cur.fetchall())


UnicodeEncodeError: 'ascii' codec can't encode characters in position 5-8: ordinal not in range(128)

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [ ]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.